In [370]:
#Important notes: All those greater than 5 had consensus.
import numpy as np
from IPython.core.display import HTML
import json
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

## THIS IS THE FINAL JUN 8 at 3
finaltaskusdf = pd.DataFrame()
finallocdf = pd.DataFrame()
data = {}
all_df= []
df_s = pd.DataFrame()
files =  ['au-extracted.json','it-extracted.json', 
          'fr-extracted.json','gb-extracted.json',
             'ca-extracted.json', 'us-extracted.json']
              

for file_num, file_name in enumerate(files):
    data = {}
    print(str(file_name))
    n=0
    with open('/Users/kimia/Desktop/Work5/sprint614/' + str(file_name)) as f:
            
                for line in f:
                    try:

                        data.update(json.loads( line.rstrip('\n')))
                        all_df.append(data)

                        taskusedits = pd.Series(data['data']['extraction']['payload']).to_frame().T
                        taskusedits['id'] = (data['uri'])
                        taskusedits['file'] = str(file_name)

                        locdf = pd.Series(data['data']['extraction']['rawPayload']).to_frame().T
                        locdf['id'] = (data['uri'])
                        locdf['file'] = str(file_name)


                        finaltaskusdf = pd.concat([taskusedits, finaltaskusdf], axis=0)
                        finallocdf = pd.concat([locdf, finallocdf], axis=0)
       
                    except:

                        n = n+1
                        print(n)
finallocdf['id'] = finallocdf['id'].apply(lambda x: int(x[-6:]))
finaltaskusdf['id']= finaltaskusdf['id'].apply(lambda x: int(x[-6:]))

## THIS WORK!! 1:36 JUN 7
data = {}
df_s = pd.DataFrame()
files = ['ca-seed-inputs.json' ,
             'it-seed-inputs.json', 'us-seed-inputs.json', 'au-seed-inputs.json', 
             'gb-seed-inputs.json' , 
             'fr-seed-inputs.json']
for file_num, file_name in enumerate(files):

    print(str(file_name)+"  i")
    with open('/Users/kimia/Desktop/Work5/sprint614/' + str(file_name)) as f:
        
        
        for line in f:
            data.update(json.loads( line.rstrip('\n')))
            #print(data['data']['extraction']['inputMeta'])

            d1 = pd.Series(data['data']['extraction']['inputMeta']).to_frame().T 
        
            d1['file'] = str(file_name)
            df_s = pd.concat([df_s, d1], axis=0)
        
result = (pd.merge(df_s,finaltaskusdf, on = ['id'])) # correct merge: (result.file_x == result.file_y)

from urllib.parse import urlparse
result['cleansite'] =result['website'].apply(lambda x: urlparse(str(x)).netloc)
result['cleansite'] =result['cleansite'].apply(lambda x: (str(x).replace("www.", "")))

au-extracted.json
it-extracted.json
fr-extracted.json
1
gb-extracted.json
1
2
3
4
5
ca-extracted.json
1
2
us-extracted.json
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
ca-seed-inputs.json  i
it-seed-inputs.json  i
us-seed-inputs.json  i
au-seed-inputs.json  i
gb-seed-inputs.json  i
fr-seed-inputs.json  i


In [161]:
result.shape

(5212, 36)

In [146]:
df2 = result.groupby('task_id').size().to_frame().reset_index()
df2[0].unique()
from collections import Counter
cnt = Counter()
for word in list(df2[0]):
    cnt[word] += 1
print('Looking at counter, I will only use tasks with 5 annotators: ', cnt)
df_5_only = result.groupby('task_id').size().to_frame().reset_index()
df_5_only = (df_5_only[df_5_only[0]==5])
dfresult = pd.merge(result, df_5_only, on = 'task_id')


Looking at counter, I will only use tasks with 5 annotators:  Counter({5: 1012, 6: 18, 4: 6, 3: 3, 2: 2, 7: 1})


In [269]:
len(df_5_only.task_id.unique())/len(result.task_id.unique())
## i dropped 3% of the dataset to make sure I was pulling accuracy correctly. 

0.9712092130518234

In [314]:
dfresult = result
def consensus(df, column):
    lowagreement = df.groupby([ 'task_id', str(column)]).size().to_frame(name = 'count').reset_index()
    agree5 = lowagreement[lowagreement['count'] >4]
    perc5 = (len(agree5.task_id.unique())/len(lowagreement.task_id.unique()))
    accuracy = ({str(column):perc5})
    return accuracy  

columnsz = [
 'address',
 'country',
 'locality',
 'name',
 'postcode',
 'region',
 'tel',
 'website']

final = pd.DataFrame()
accuracydict={}
for number,country in enumerate(result.file_x.unique()):
    
    for i, v in enumerate(columnsz): 


        percent = (consensus(dfresult[dfresult.file_x ==str(country)], str(v)))
        accuracydict.update(percent)

    all_perc_disagree = pd.Series(accuracydict).to_frame().sort_values([0],ascending = True)
    all_perc_disagree.columns = [str(country[0:2])]
    final = pd.concat([all_perc_disagree, final], axis=1)
    #print("% disagreement by column in ",str(country[0:2]) )
print("    MODERATION: % there is consensus among 5 on a POI, by field input")
display(HTML(((final).to_html())))


    MODERATION: % there is consensus among 5 on a POI, by field input


,fr,gb,au,us,it,ca
address,0.384000,0.208696,0.622951,0.553846,0.384615,0.468750
country,0.978261,0.985075,1.000000,0.996885,0.973958,1.000000
locality,0.884058,0.932331,0.849462,0.937695,0.880208,1.000000
name,0.699248,0.773438,0.831579,0.800000,0.562500,0.712121
postcode,0.905797,0.902985,0.888298,0.859813,0.750000,0.848485
region,0.978261,0.947761,1.000000,0.993769,0.942708,1.000000
tel,0.736842,0.686567,0.767196,0.640625,0.434066,0.761905
website,0.147287,0.203008,0.308511,0.196825,0.142857,0.218750


In [304]:
dfresult['cleantel'] = dfresult['tel'].apply(lambda x: ((str(x).replace(" ", ""))))

final = pd.DataFrame()
accuracydict={}

columnsz = ['cleansite','website']
for number,country in enumerate(result.file_x.unique()):
    
    for i, v in enumerate(columnsz): 


        percent = (consensus(dfresult[dfresult.file_x ==str(country)], str(v)))
        accuracydict.update(percent)

    all_perc_disagree = pd.Series(accuracydict).to_frame().sort_values([0],ascending = True)
    all_perc_disagree.columns = [str(country[0:2])]
    final = pd.concat([all_perc_disagree, final], axis=1)
    #print("% disagreement by column in ",str(country[0:2]) )
print("    MODERATION: website has sig improvement with urllib parser")
display(HTML(((final).to_html())))


    MODERATION: website has sig improvement with urllib parser


,fr,gb,au,us,it,ca
website,0.147287,0.203008,0.308511,0.196825,0.142857,0.218750
cleansite,0.536232,0.664179,0.747368,0.623053,0.437500,0.686567


In [313]:
dfresult['cleantel'] = dfresult['tel'].apply(lambda x: ((str(x).replace(" ", ""))))

final = pd.DataFrame()
accuracydict={}

columnsz = ['cleantel','tel']
for number,country in enumerate(result.file_x.unique()):
    
    for i, v in enumerate(columnsz): 


        percent = (consensus(dfresult[dfresult.file_x ==str(country)], str(v)))
        accuracydict.update(percent)

    all_perc_disagree = pd.Series(accuracydict).to_frame().sort_values([0],ascending = True)
    all_perc_disagree.columns = [str(country[0:2])]
    final = pd.concat([all_perc_disagree, final], axis=1)
    #print("% disagreement by column in ",str(country[0:2]) )
print("    MODERATION: Tel cannot be saved in Italy by removing spaces")
display(HTML(((final).to_html())))


    MODERATION: Tel cannot be saved in Italy by removing spaces


,fr,gb,au,us,it,ca
cleantel,0.746377,0.686567,0.768421,0.641745,0.463542,0.776119
tel,0.736842,0.686567,0.767196,0.640625,0.434066,0.761905


In [321]:
#(result[result.file_x == 'it-seed-inputs.json']).address

In [375]:
sum((result[['address', 'tel', 'website', 'name']].isnull().sum(axis=1)))/result.shape[0]

0.44013814274750573

In [360]:
kimia = result
kimia['tel'] = kimia['tel'].fillna('KIMIA')

ff = result.groupby([ 'task_id', 'tel']).size().to_frame(name = 'count').reset_index()
dfwithnull = ff[ff['tel'] == "KIMIA"]
list(dfwithnull.task_id)

df_without_null = result
for i,v in enumerate(list(dfwithnull.task_id)): 
    df_without_null = df_without_null[df_without_null.task_id != v]
len(df_without_null.task_id.unique())/len(result.task_id.unique())
df_without_null[df_without_null.task_id ==59039]

,id,sourceUrl,task_id,user_id,file_x,_original_category,_records_touched,address,address_accuracy,address_extended,...,post_town,postcode,raw_address,rbdi,region,tel,type,website,cleansite,cleantel


In [381]:
final = pd.DataFrame()
accuracydict={}

columnsz = [
 'address',
 'country',
 'locality',
 'name',
 'postcode',
 'region',
 'tel',
 'website']
listy2 = []
for number,country in enumerate(result.file_x.unique()):
    
    for i, v in enumerate(columnsz): 
        
        #get a list of all task IDs with Null
        removeNA = result
        removeNA[str(v)] = removeNA[str(v)].fillna('KIMIA')

        ff = result.groupby([ 'task_id', str(v)]).size().to_frame(name = 'count').reset_index()
        dfwithnull = ff[ff[str(v)] == "KIMIA"]

        ## now have a fresh df, 
        df_without_null = result
        for num,task_id in enumerate(list(dfwithnull.task_id)): 
            df_without_null = df_without_null[df_without_null.task_id != (task_id)]
        #(rr.tel).isnull().sum(axis=0)
        percent = (consensus(df_without_null[df_without_null.file_x ==str(country)], str(v)))
        countrydf = df_without_null[df_without_null.file_x ==str(country)]
        countryfulldf = result[result.file_x ==str(country)]
        listy = list([country, v, len(countrydf.task_id.unique()), len(countryfulldf.task_id.unique()), \
              (len(countrydf.task_id.unique())/len(countryfulldf.task_id.unique()))])
        accuracydict.update(percent)
        listy2.append(listy)
    all_perc_disagree = pd.Series(accuracydict).to_frame().sort_values([0],ascending = True)
    all_perc_disagree.columns = [str(country[0:2])]
    final = pd.concat([all_perc_disagree, final], axis=1)
    #print("% disagreement by column in ",str(country[0:2]) )
    
print("    MODERATION: REMOVING NULL")
display(HTML(((final).to_html())))

#Here I dropped any task_Id where there was an NA--> which would have obviously dropped 

    MODERATION: REMOVING NULL


,fr,gb,au,us,it,ca
address,0.489796,0.489796,0.735484,0.831395,0.500000,0.555556
country,0.978261,0.985075,1.000000,0.996885,0.973958,1.000000
locality,0.938462,0.953846,0.918605,0.937695,0.903743,1.000000
name,0.715385,0.779528,0.831579,0.810127,0.568421,0.758065
postcode,0.905797,0.902985,0.943503,0.865204,0.753927,0.875000
region,0.978261,0.962121,1.000000,0.993769,0.942708,1.000000
tel,0.837607,0.739837,0.843023,0.750916,0.600000,0.842105
website,0.218391,0.234783,0.343195,0.248000,0.268817,0.245614


In [385]:
concerndetect = pd.DataFrame(listy2, columns = ['country', 'attribute', 'num', 'denom', 'perc'])
concerndetect.sort_values(['perc'],ascending = True)


,country,attribute,num,denom,perc
32,gb-seed-inputs.json,address,49,134,0.365672
15,it-seed-inputs.json,website,93,192,0.484375
16,us-seed-inputs.json,address,172,321,0.535826
47,fr-seed-inputs.json,website,87,138,0.630435
14,it-seed-inputs.json,tel,130,192,0.677083
40,fr-seed-inputs.json,address,98,138,0.710145
8,it-seed-inputs.json,address,140,192,0.729167
23,us-seed-inputs.json,website,250,321,0.778816
0,ca-seed-inputs.json,address,54,67,0.805970
24,au-seed-inputs.json,address,155,190,0.815789


In [ ]:
## what happens if I remove all tasks with Nan in important fields


In [227]:
## can we clean up postcode? NO.
listpost = (dfresult[dfresult.file_x == 'it-seed-inputs.json']['tel'].apply(lambda x: len(str(x))))
cnt = Counter()
for word in list(listpost):
    cnt[word] += 1
print('Postcode has issues other than cleaning ', cnt)

Postcode has issues other than cleaning  Counter({12: 314, 11: 186, 3: 144, 10: 131, 9: 85, 7: 29, 13: 18, 8: 18})


In [259]:
#import replace

listpost = dfresult[dfresult.file_x == 'it-seed-inputs.json']['tel'].apply(lambda x: ((str(x).replace(" ", ""))))
cnt = Counter()
for word in list(listpost):
    cnt[word] += 1
print('Italian Phone numbers have issues other than cleaning: It is not the country code that is messing up the dataset\
      Italy uses an open numbering \
plan in which the length of subscriber telephone numbers varies from six to eleven digits. \
The dialing procedures for callers mandate that the full telephone number is always dialed, \
including the area code')


Italian Phone numbers have issues other than cleaning: It is not the country code that is messing up the dataset      Italy uses an open numbering plan in which the length of subscriber telephone numbers varies from six to eleven digits. The dialing procedures for callers mandate that the full telephone number is always dialed, including the area code


In [260]:
listpost

330      0642740571
331      0642740571
332      0642740571
333      0642740571
334      0642740571
335      0417241040
336      0417241040
337      0415298711
338      0417241040
339             nan
340      0577940328
341      0577940008
342      0577940328
343      0577940328
344      0577940328
345        04817731
346      0481773224
347      0481773224
348      0481773224
349      0481773224
350      0916404403
351      0916404403
352      0916404403
353      0916404403
354      0916404403
355       011090102
356       011090102
357       011090102
358       011090102
359      0110906111
           ...     
1225     0172741500
1226     0172741500
1227     0172741500
1228     0172741500
1229     0172741500
1230       05219515
1231     0521951511
1232     0521951511
1233     0521951511
1234       05219515
1235     0805754201
1236      098427485
1237    09841811234
1238     0805213704
1239     0805213704
1240      075812301
1241      075812301
1242      075812301
1243      075812301


In [85]:
# Top Performers

def consensus3(df, column):
    lowagreement = df.groupby([ 'task_id', str(column)]).size().to_frame(name = 'count').reset_index()
    agree3 = lowagreement[lowagreement['count'] >2]
    return agree3

In [86]:
perc5 = []
perc3 = []
names  = []
for i, v in enumerate(['name', 'tel', 'website', 'cleansite','address']):
    names.append(str(v))
    agree5 = consensus(df, str(v))
    perc5.append(len(agree5.task_id.unique())/len(result.task_id.unique()))
    agree3 = consensus3(df, str(v))
    perc3.append(len(agree3.task_id.unique())/len(result.task_id.unique()))

print('% of time there is consensus among X annotators')
pd.DataFrame({  'attribute': names,'5annot': perc5,})

% of time there is consensus among X annotators


,5annot,attribute
0,0.712092,name
1,0.622841,tel
2,0.187140,website
3,0.544146,cleansite
4,0.401152,address


In [131]:
for i, v in enumerate(columnsz): 

    print(consensus(df, column))

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [140]:
df2 = result
def accuracy_per_column(df, column):
    cons = df.groupby([ 'task_id', (str(column))]).size().to_frame()
    cons['count'] = cons[0]
    j = (cons[cons['count']<5]).reset_index()
    numer_mistakes  = len(j['task_id'].unique())
    total = len(df['task_id'].unique())
    accuracy = ({str(column):numer_mistakes/total})
    return accuracy
    ### make this a pretty print
    
def consensus(df, column):
    lowagreement = df.groupby([ 'task_id', str(column)]).size().to_frame(name = 'count').reset_index()
    agree5 = lowagreement[lowagreement['count'] >4]
    perc5 = (len(agree5.task_id.unique())/len(result.task_id.unique()))
    accuracy = ({str(column):perc5})
    return accuracy
    
columnsz = [
 'address',
 'address_extended',
 'country',
 'locality',
 'name',
 'postcode',
 'region',
 'tel', 'cleansite',
 'website']
    
accuracydict={}



final = pd.DataFrame()
for number,country in enumerate(df.file_x.unique()):
    df = df2[df2.file_x == str(country)]
    
    for i, v in enumerate(columnsz): 
        accuracydict.update(consensus(df, str(v)))
    all_perc_disagree = pd.Series(accuracydict).to_frame().sort_values([0],ascending = True)
    all_perc_disagree.columns = [str(country[0:2])]
    final = pd.concat([all_perc_disagree, final], axis=1)
    #print("% disagreement by column in ",str(country[0:2]) )
print("    % there is not consensus on a POI, by field input")
display(HTML(((final).to_html())))

    % there is not consensus on a POI, by field input


""


In [205]:
pe =dfresult[dfresult.file_x ==str('fr-seed-inputs.json')]
lowagreement =pe.groupby([ 'task_id', str('address_extended')]).size().to_frame(name = 'count').reset_index()
agree5 = lowagreement[lowagreement['count'] >4]

print(len(agree5.task_id.unique())/len(lowagreement.task_id.unique()))
#accuracy = ({str(column):perc5})

ZeroDivisionError: division by zero

In [211]:
def consensus(df, column):
    lowagreement = df.groupby([ 'task_id', str(column)]).size().to_frame(name = 'count').reset_index()
    agree5 = lowagreement[lowagreement['count'] >4]
    perc5 = (len(agree5.task_id.unique())/len(lowagreement.task_id.unique()))
    accuracy = ({str(column):perc5})
    return accuracy  

columnsz = [
 'address',
 'country',
 'locality',
 'name',
 'postcode',
 'region',
 'tel',
    'cleansite',
 'website']

final = pd.DataFrame()
accuracydict={}
for number,country in enumerate(result.file_x.unique()):
    
    for i, v in enumerate(columnsz): 


        percent = (consensus(dfresult[dfresult.file_x ==str(country)], str(v)))
        accuracydict.update(percent)

    all_perc_disagree = pd.Series(accuracydict).to_frame().sort_values([0],ascending = True)
    all_perc_disagree.columns = [str(country[0:2])]
    final = pd.concat([all_perc_disagree, final], axis=1)
    #print("% disagreement by column in ",str(country[0:2]) )
print("    MODERATION: % there is consensus on a POI, by field input")
display(HTML(((final).to_html())))

    MODERATION: % there is consensus on a POI, by field input


,fr,gb,au,us,it,ca
address,0.391667,0.207207,0.620112,0.547244,0.394286,0.460317
cleansite,0.507576,0.635659,0.682796,0.541401,0.432432,0.621212
country,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
locality,0.901515,0.945312,0.846154,0.939490,0.908108,1.000000
name,0.708661,0.780488,0.827957,0.798722,0.572973,0.707692
postcode,0.924242,0.914729,0.885870,0.859873,0.772973,0.846154
region,1.000000,0.961240,1.000000,0.996815,0.967568,1.000000
tel,0.748031,0.689922,0.762162,0.638978,0.440000,0.758065
website,0.146341,0.203125,0.309783,0.185065,0.142857,0.206349


In [177]:

def accuracy_per_column(df, column):
    cons = df.groupby([ 'task_id', (str(column))]).size().to_frame()
    cons['count'] = cons[0]
    j = (cons[cons['count']<5]).reset_index()
    numer_mistakes  = len(j['task_id'].unique())
    total = len(df['task_id'].unique())
    accuracy = ({str(column):numer_mistakes/total})
    return accuracy
    ### make this a pretty print
def consensus(df, column):
    lowagreement = df.groupby([ 'task_id', str(column)]).size().to_frame(name = 'count').reset_index()
    agree5 = lowagreement[lowagreement['count'] >4]
    perc5 = (len(agree5.task_id.unique())/len(lowagreement.task_id.unique()))
    accuracy = ({str(column):perc5})
    return accuracy  
    
columnsz = [
 'address',
 'address_extended',
 'country',
 'locality',
 'name',
 'postcode',
 'region',
 'tel',
 'website']
    


final = pd.DataFrame()
accuracydict={}
for number,country in enumerate(result.file_x.unique()):
    df = dfresult[dfresult.file_x == str(country)]
    
    for i, v in enumerate(columnsz): 
        accuracydict.update(consensus(df, str(v)))
    all_perc_disagree = pd.Series(accuracydict).to_frame().sort_values([0],ascending = True)
    all_perc_disagree.columns = [str(country[0:2])]
    final = pd.concat([all_perc_disagree, final], axis=1)
    #print("% disagreement by column in ",str(country[0:2]) )
print("    % there is not consensus on a POI, by field input")
display(HTML(((final).to_html())))

ZeroDivisionError: division by zero

In [115]:
sample=2
column = 'tel'
dfsample4 = df.groupby('task_id').apply(lambda x: (x.sample(int(sample)))).reset_index(drop=True)
# make duplicate means 1 if dup, else = 0
dfsample4["dup"] = 0
#dfsample4.loc[dfsample4['annotation'] == 'duplicate', 'dup'] = 1
# get the ratio of consensus
#prob = dfsample4.groupby('unique').apply(lambda x: (np.sum(x['dup'])/int(sample))).reset_index(drop=True)
sample2 = (dfsample4).groupby(['task_id', str(column)]).size().to_frame(name = 'count').reset_index()
sample2consensus = (sample2[sample2['count'] == sample])
sample2consensus.shape[0]/len(df['task_id'].unique())

0.7737154150197628

In [ ]:


## take a sample
dfsample4 = df.groupby('unique').apply(lambda x: (x.sample(int(sample)))).reset_index(drop=True)
    # make duplicate means 1 if dup, else = 0
    dfsample4["dup"] = 0
    dfsample4.loc[dfsample4['annotation'] == 'duplicate', 'dup'] = 1
    # get the ratio of consensus
    prob = dfsample4.groupby('unique').apply(lambda x: (np.sum(x['dup'])/int(sample))).reset_index(drop=True)

    # only where everyone totally agrees. 
    probabilities = pd.DataFrame(prob)
    probabilities['sample'] = int(sample)
    probabilities['consensus'] = 0
    probabilities.loc[probabilities[0] == 0, 'consensus'] = 1
    probabilities.loc[probabilities[0] == 1, 'consensus'] = 1
    all_probabilities = pd.concat([all_probabilities, probabilities], axis = 0)



In [15]:
df = result
agree5 = consensus(df, 'tel')
agree5
j = pd.merge(agree5, result, how ="right", on=[ 'task_id'])


,task_id,tel_x,count,id,sourceUrl,user_id,file_x,_original_category,_records_touched,address,...,og_address,po_box,post_town,postcode,raw_address,rbdi,region,tel_y,type,website
0,59033.0,(08) 9168 7300,5.0,167386,https://yny.com/167386,16,au-seed-inputs.json,{'mapping_not_necessary': ['119']},crawl,NaN,...,NaN,NaN,NaN,6770,NaN,NaN,WA,(08) 9168 7300,NaN,https://parks.dpaw.wa.gov.au/park/purnululu
1,59033.0,(08) 9168 7300,5.0,167014,https://yny.com/167014,59,au-seed-inputs.json,{'mapping_not_necessary': ['119']},crawl,NaN,...,NaN,NaN,NaN,6770,NaN,NaN,WA,(08) 9168 7300,NaN,http://parks.dpaw.wa.gov.au/park/purnululu
2,59033.0,(08) 9168 7300,5.0,167013,https://yny.com/167013,21,au-seed-inputs.json,{'mapping_not_necessary': ['119']},crawl,NaN,...,NaN,NaN,NaN,6770,NaN,NaN,WA,(08) 9168 7300,NaN,http://parks.dpaw.wa.gov.au/park/purnululu
3,59033.0,(08) 9168 7300,5.0,167010,https://yny.com/167010,31,au-seed-inputs.json,{'mapping_not_necessary': ['119']},crawl,NaN,...,NaN,NaN,NaN,6770,NaN,NaN,WA,(08) 9168 7300,NaN,http://parks.dpaw.wa.gov.au/park/purnululu
4,59033.0,(08) 9168 7300,5.0,165681,https://yny.com/165681,39,au-seed-inputs.json,{'mapping_not_necessary': ['119']},crawl,NaN,...,NaN,NaN,NaN,6770,NaN,NaN,WA,(08) 9168 7300,NaN,http://parks.dpaw.wa.gov.au/park/purnululu
5,59035.0,(780) 425-5379,5.0,165696,https://yny.com/165696,21,ca-seed-inputs.json,{'mapping_not_necessary': ['310']},crawl,2 Sir Winston Churchill Sq,...,NaN,NaN,NaN,T5J 2C1,NaN,NaN,AB,(780) 425-5379,NaN,http://www.youraga.ca
6,59035.0,(780) 425-5379,5.0,165700,https://yny.com/165700,39,ca-seed-inputs.json,{'mapping_not_necessary': ['310']},crawl,2 Sir Winston Churchill Sq NW,...,NaN,NaN,NaN,T5J 2C1,NaN,NaN,AB,(780) 425-5379,NaN,https://www.youraga.ca/
7,59035.0,(780) 425-5379,5.0,165699,https://yny.com/165699,47,ca-seed-inputs.json,{'mapping_not_necessary': ['310']},crawl,2 Sir Winston Churchill Sq NW,...,NaN,NaN,NaN,T5J 2C1,NaN,NaN,AB,(780) 425-5379,NaN,http://www.youraga.ca
8,59035.0,(780) 425-5379,5.0,165695,https://yny.com/165695,36,ca-seed-inputs.json,"{'mapped': [], 'mappings': {}, 'unmapped': ['3...",crawl,2 Sir Winston Churchill Sq,...,NaN,NaN,NaN,T5J 2C1,NaN,NaN,AB,(780) 425-5379,NaN,http://www.youraga.ca
9,59035.0,(780) 425-5379,5.0,165693,https://yny.com/165693,46,ca-seed-inputs.json,{'mapping_not_necessary': ['310']},crawl,2 Sir Winston Churchill Sq,...,NaN,NaN,NaN,T5J 2C1,NaN,NaN,AB,(780) 425-5379,NaN,http://www.youraga.ca


In [19]:
len(agree5.task_id.unique())/len(result.task_id.unique())

0.6401151631477927

In [ ]:
df = result 
total=pd.DataFrame()
for i, column in enumerate(['name', 'tel', 'website', 'address']):
    
    lowagreement = df.groupby([ 'task_id', str(column)]).size().to_frame(name = 'count').reset_index()
    agree5 = lowagreement[lowagreement['count'] >4]
    
    agree5['CONSENSUS'] = 1
    j = pd.merge(agree5, result, how ="right", on=[ 'task_id'])
    j['CONSENSUS'] = j['CONSENSUS'].fillna(0)
    
    total = pd.concat([total,j], axis = 1 )
    total = total.rename(columns={"CONSENSUS": str(column)+'CONSENSUS'})

    total
total['country'] = total['file_x'][0:2]